In [11]:
import tensorflow.keras as keras
import numpy as np
from functools import partial

In [47]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.mnist.load_data()

In [48]:
x_train_full = x_train_full / 255.
x_test = x_test/255.

In [49]:
x_train, x_valid = x_train_full[:-5000], x_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [50]:
x_train = x_train[..., np.newaxis]
x_valid = x_valid[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [12]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3,
                        padding='same', activation='relu', kernel_initializer='he_normal')

In [16]:
model = keras.Sequential([
    DefaultConv2D(filters=64, input_shape=[28, 28, 1]),
    DefaultConv2D(filters=64, input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=120, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.7),
    keras.layers.Dense(10, activation='softmax')
])

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [18]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 293s 169ms/step - loss: 2.3017 - accuracy: 0.1107 - val_loss: 2.3019 - val_accuracy: 0.1060
Epoch 2/5
1719/1719 [==============================] - 285s 166ms/step - loss: 2.3013 - accuracy: 0.1129 - val_loss: 2.3018 - val_accuracy: 0.1060
Epoch 3/5
1719/1719 [==============================] - 302s 176ms/step - loss: 2.3013 - accuracy: 0.1129 - val_loss: 2.3018 - val_accuracy: 0.1060
Epoch 4/5
1719/1719 [==============================] - 330s 192ms/step - loss: 2.3013 - accuracy: 0.1129 - val_loss: 2.3018 - val_accuracy: 0.1060
Epoch 5/5
1719/1719 [==============================] - 328s 191ms/step - loss: 2.3013 - accuracy: 0.1129 - val_loss: 2.3019 - val_accuracy: 0.1060


## Things to be done to the model
The following model uses 2 convolutional layers, followed by 1 pooling layer, then dropout 25%, then a dense layer, another dropout layer but with 50% dropout, and finally the output layer. It reaches about 99.2% accuracy on the test set. This places this model roughly in the top 20% in the MNIST Kaggle competition (if we ignore the models with an accuracy greater than 99.79% which were most likely trained on the test set, as explained by Chris Deotte in this post). Can you do better? To reach 99.5 to 99.7% accuracy on the test set, you need to add image augmentation, batch norm, use a learning schedule such as 1-cycle, and possibly create an ensemble.

In [20]:
model_second_gen = keras.Sequential([
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=64),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='relu')
])

In [21]:
model_second_gen.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics='accuracy')

In [23]:
history = model_second_gen.fit(x_train, y_train, epochs = 10, validation_data=(x_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 239s 138ms/step - loss: 2.3746 - accuracy: 0.1997 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 2/10
1719/1719 [==============================] - 307s 178ms/step - loss: 2.3138 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 3/10
1719/1719 [==============================] - 3874s 2s/step - loss: 2.3085 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 4/10
1719/1719 [==============================] - 347s 202ms/step - loss: 2.3049 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 5/10
1719/1719 [==============================] - 1011s 588ms/step - loss: 2.3060 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 6/10
1719/1719 [==============================] - 308s 179ms/step - loss: 2.3040 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.0978
Epoch 7/10
1719/1719 [==============================] - 293s 170ms/step - loss: 2.3029 - accuracy: 0.0988 - val_l

In [25]:
model_second_gen.evaluate(x_test, y_test)

313/313 [==============================] - 10s 31ms/step - loss: 2.3026 - accuracy: 0.0980


[2.30259108543396, 0.09799999743700027]

In [26]:
third_model = keras.Sequential([
    DefaultConv2D(filters=32, kernel_size=7),
    DefaultConv2D(filters=32, kernel_size=7),
    keras.layers.MaxPool2D(),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
    
])

In [27]:
third_model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

Using the resnet model 

In [53]:
x_train = x_train.repeat(3, axis=-1)

In [55]:
x_valid = x_valid.repeat(3, axis=-1)

In [56]:
x_test = x_test.repeat(3, axis=-1)

In [57]:
x_train = keras.applications.resnet50.preprocess_input(x_train)
x_valid = keras.applications.resnet50.preprocess_input(x_valid)

In [59]:
x_test = keras.applications.resnet50.preprocess_input(x_test)

In [60]:
base_model = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAvgPool2D()(base_model.output)
output = keras.layers.Dense(10, activation='softmax')(avg)
fourth_model = keras.Model(inputs=base_model.input, outputs=output)

In [61]:
for layer in base_model.layers:
    layer.trainable = False

In [62]:
optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
fourth_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [63]:
history = fourth_model.fit(x_train, y_train, epochs=3, validation_data=(x_valid, y_valid))

Epoch 1/3
1719/1719 [==============================] - 162s 92ms/step - loss: 2.1313 - accuracy: 0.2405 - val_loss: 12.6830 - val_accuracy: 0.1060
Epoch 2/3
1719/1719 [==============================] - 155s 90ms/step - loss: 1.7612 - accuracy: 0.4634 - val_loss: 22.1345 - val_accuracy: 0.1060
Epoch 3/3
1719/1719 [==============================] - 154s 90ms/step - loss: 1.5124 - accuracy: 0.5671 - val_loss: 29.1784 - val_accuracy: 0.1062


In [64]:
history = fourth_model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 145s 84ms/step - loss: 1.3368 - accuracy: 0.6268 - val_loss: 34.3090 - val_accuracy: 0.1120
Epoch 2/5
1719/1719 [==============================] - 151s 88ms/step - loss: 1.2071 - accuracy: 0.6671 - val_loss: 38.7660 - val_accuracy: 0.1214
Epoch 3/5
1719/1719 [==============================] - 151s 88ms/step - loss: 1.1086 - accuracy: 0.6991 - val_loss: 42.9783 - val_accuracy: 0.1256
Epoch 4/5
1719/1719 [==============================] - 152s 88ms/step - loss: 1.0298 - accuracy: 0.7197 - val_loss: 46.7023 - val_accuracy: 0.1340
Epoch 5/5
1719/1719 [==============================] - 149s 87ms/step - loss: 0.9653 - accuracy: 0.7349 - val_loss: 50.2427 - val_accuracy: 0.0908


In [65]:
history = fourth_model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
1719/1719 [==============================] - 141s 82ms/step - loss: 0.9130 - accuracy: 0.7524 - val_loss: 55.0290 - val_accuracy: 0.0998
Epoch 2/20
1719/1719 [==============================] - 134s 78ms/step - loss: 0.8693 - accuracy: 0.7627 - val_loss: 59.1325 - val_accuracy: 0.0996
Epoch 3/20
1719/1719 [==============================] - 157s 91ms/step - loss: 0.8319 - accuracy: 0.7709 - val_loss: 63.8977 - val_accuracy: 0.1034
Epoch 4/20
1719/1719 [==============================] - 136s 79ms/step - loss: 0.7997 - accuracy: 0.7813 - val_loss: 68.6919 - val_accuracy: 0.1030
Epoch 5/20
1719/1719 [==============================] - 139s 81ms/step - loss: 0.7703 - accuracy: 0.7878 - val_loss: 73.5591 - val_accuracy: 0.1014
Epoch 6/20
1719/1719 [==============================] - 142s 83ms/step - loss: 0.7456 - accuracy: 0.7943 - val_loss: 78.5232 - val_accuracy: 0.1002
Epoch 7/20
1719/1719 [==============================] - 3045s 2s/step - loss: 0.7233 - accuracy: 0.8006 - val_lo

KeyboardInterrupt: 